In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ptflops import get_model_complexity_info
from einops import rearrange, repeat

from CrossAttn import CrossAttn
from deepSense import DeepSense
from GlobalFusion import GlobalFusion
from ALAE import ALAE_TAE
from Attend import AttendDiscriminate
from deepconvlstm import DeepConvLSTM
from ifconv import If_ConvTransformer_W
from TinyHAR import TinyHAR_Model
from MixerMLP import FFTMIXER_HAR_Model

In [2]:
data_dict = [
    # {"dataset_name":"pamap2",  "sensor_channel":18, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":18, "fft_feature_dim":40, #
    #  "S_number_sensors_type":2,"L_sensor_locations":3},#
    # {"dataset_name":"dsads",  "sensor_channel":45, "window_length":125, #
    #  "fft_window_length":5, "nb_classes":19, "fft_feature_dim":50, #
    #  "S_number_sensors_type":3,"L_sensor_locations":5},#
    # {"dataset_name":"dg",  "sensor_channel":9, "window_length":128, #
    #  "fft_window_length":8, "nb_classes":2, "fft_feature_dim":32, #
    #  "S_number_sensors_type":1,"L_sensor_locations":3},#
    # {"dataset_name":"rw",  "sensor_channel":45, "window_length":100, #
    #  "fft_window_length":10, "nb_classes":8, "fft_feature_dim":20, #
    #  "S_number_sensors_type":3,"L_sensor_locations":5},#
    # {"dataset_name":"hapt",  "sensor_channel":9, "window_length":128, #
    #  "fft_window_length":8, "nb_classes":12, "fft_feature_dim":32, #
    #  "S_number_sensors_type":3,"L_sensor_locations":1},#
    # {"dataset_name":"motionsense",  "sensor_channel":12, "window_length":128, #
    #  "fft_window_length":8, "nb_classes":6, "fft_feature_dim":32, 
    #  "S_number_sensors_type":4,"L_sensor_locations":1},#
    # {"dataset_name":"skoda",  "sensor_channel":30, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":10, "fft_feature_dim":40, #
    #  "S_number_sensors_type":1,"L_sensor_locations":10},#
    # {"dataset_name":"GesHome",  "sensor_channel":9, "window_length":50,# 
    #  "fft_window_length":7, "nb_classes":18, "fft_feature_dim":20, #
    #  "S_number_sensors_type":3,"L_sensor_locations":1},#
    # {"dataset_name":"lsign",  "sensor_channel":15, "window_length":75, #
    #  "fft_window_length":6, "nb_classes":36, "fft_feature_dim":40, #
    #  "S_number_sensors_type":1,"L_sensor_locations":5},#
    # {"dataset_name":"emgg",  "sensor_channel":9, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":7, "fft_feature_dim":40, #
    #  "S_number_sensors_type":1,"L_sensor_locations":3},#
    # {"dataset_name":"hhar",  "sensor_channel":6, "window_length":200, #
    #  "fft_window_length":10, "nb_classes":6, "fft_feature_dim":40, #
    #  "S_number_sensors_type":2,"L_sensor_locations":1},#
    {"dataset_name":"mhealth",  "sensor_channel":18, "window_length":128, #
     "fft_window_length":8 , "nb_classes":12, "fft_feature_dim":32, #
     "S_number_sensors_type":3,"L_sensor_locations":2}#
]


In [7]:
results={}
for data in data_dict:
    #print(data)
    result_data = {}
    dataset_name=data["dataset_name"]
    sensor_channel=data["sensor_channel"]
    window_length=data["window_length"]
    fft_window_length=data["fft_window_length"]
    nb_classes=data["nb_classes"]
    fft_feature_dim=data["fft_feature_dim"]
    S_number_sensors_type=data["S_number_sensors_type"]
    L_sensor_locations=data["L_sensor_locations"]
    
    
    #  ----------------  build DCL model ---------------------

    config = {  "nb_conv_blocks": 2,
      "nb_filters": 64,
      "dilation": 1,
      "batch_norm": 0,
      "filter_width": 7,
      "nb_layers_lstm": 1,
      "drop_prob": 0.2,
      "nb_units_lstm": 128}
    input_shape = (1, 1,window_length,sensor_channel)
    model = DeepConvLSTM(input_shape, nb_classes, 1,config=config)
    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)

    result_data["dcl_macs"] = macs
    result_data["dcl_params"] = params
    # -------------- build DeepSense Model 

    if fft_feature_dim<=21:
        kernel_size = 3
    else:
        kernel_size = 5
    model = DeepSense(input_shape=(1, 1, window_length, sensor_channel),   fft_segments_length = int(fft_feature_dim/2),   
                      k_number_sensors_group=S_number_sensors_type*L_sensor_locations, nb_classes= nb_classes, kernel_size_1=kernel_size)

    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)
    result_data["DeepSense_macs"] = macs
    result_data["DeepSense_params"] = params

    # ==============build  Globalfusion model 

    model = GlobalFusion(input_shape=(1, 1, window_length, sensor_channel),      fft_segments_length = int(fft_feature_dim/2),   
                         S_number_sensors_type=S_number_sensors_type, 
                         L_sensor_locations= L_sensor_locations, nb_classes= nb_classes, kernel_size_1 = kernel_size)


    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=True)

    result_data["GlobalFusion_macs"] = macs
    result_data["GlobalFusion_params"] = params

    # ---------------- Build Attend
    input_shape = (1, 1,window_length,sensor_channel)
    config = {"hidden_dim": 128,
      "filter_num": 64,
      "filter_size": 5,
      "enc_num_layers": 2,
      "dropout": 0.5,
      "dropout_rnn": 0.25,
      "dropout_cls": 0.5,
      "activation": "ReLU",
      "sa_div": 1}
    model = AttendDiscriminate(input_shape, nb_classes, config=config)
    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["Attend_macs"] = macs
    result_data["Attend_params"] = params
    # ---------- Build ALAE ----------


    model = ALAE_TAE(input_shape=(1, 1, window_length, sensor_channel),      
                     nb_classes=nb_classes)

    #x = torch.rand(1, 1, window_length, sensor_channel)
    macs, params = get_model_complexity_info(model, 
                                             (1, window_length, sensor_channel), 
                                             as_strings=True, print_per_layer_stat=False, verbose=False)


    result_data["ALAE_macs"] = macs
    result_data["ALAE_params"] = params
    # ---------Build  IF Conv --------------


    
    if sensor_channel%9==0:
        
        fusion = True
    else:
        fusion = False
    model = If_ConvTransformer_W(1, sensor_channel, 32, 5, 3, 2, 64, 1, 0.2, dataset_name, window_length, nb_classes, fusion = fusion)
    
    #input = torch.randn(1, 1,sensor_channel , window_length)

    
    
    macs, params = get_model_complexity_info(model, (1,sensor_channel , window_length), as_strings=True,
                                           print_per_layer_stat=False, verbose=False)


    result_data["IfConv_macs"] = macs
    result_data["IfConv_params"] = params
    # --------------- Build cross Attention -------------


    # model = CrossAttn(
    #     Ts_input_shape = (1,1,window_length,sensor_channel),
    #     hidden_dim   = 24,
    #     FFT_input_shape = (1,fft_feature_dim,fft_window_length,sensor_channel)
    # )


    # def prepare_input(resolution):
    #     x1 = torch.FloatTensor(1, 1, window_length, sensor_channel)
    #     x2 = torch.FloatTensor(1, fft_feature_dim, fft_window_length, sensor_channel)
    #     return dict(x = [x1, x2])



    # macs, params = get_model_complexity_info(model, 
    #                                          (1, window_length, sensor_channel), 
    #                                          input_constructor=prepare_input, 
    #                                          as_strings=True, print_per_layer_stat=False, verbose=False)

    # result_data["CrossAttn_macs"] = macs
    # result_data["CrossAttn_params"] = params


    

    # results[dataset_name] = result_data

    # ----------------- build the Mixer Model -------------

    config["fft_mixer_share_flag"] = False
    config["fft_mixer_temporal_flag"]  = True
    config["fft_mixer_FFT_flag"]  = True
    model  = FFTMIXER_HAR_Model(input_shape=(1,1, window_length, sensor_channel ), 
                                number_class = nb_classes,
                                filter_num = 6,
                                fft_mixer_segments_length = int(fft_feature_dim/2),
                                expansion_factor = 0.3,
                                fft_mixer_layer_nr = 2,
                                fuse_early = False,
                                temporal_merge= True,
                                oration = 0.25,
                                model_config = config)
    
    macs, params = get_model_complexity_info(model, (1,window_length , sensor_channel), as_strings=True,
                                           print_per_layer_stat=False, verbose=False)


    result_data["Mixer_macs"] = macs
    result_data["Mixer_params"] = params

 self.nr_segment : 11
-----------use fusion -----


In [6]:
fft_feature_dim

32

In [8]:
result_data

{'dcl_macs': '241.26 MMac',
 'dcl_params': '744.65 k',
 'DeepSense_macs': '45.11 MMac',
 'DeepSense_params': '325.26 k',
 'GlobalFusion_macs': '38.76 MMac',
 'GlobalFusion_params': '333.64 k',
 'Attend_macs': '226.53 MMac',
 'Attend_params': '667.34 k',
 'ALAE_macs': '190.41 MMac',
 'ALAE_params': '413.13 k',
 'IfConv_macs': '87.39 MMac',
 'IfConv_params': '170.74 k',
 'Mixer_macs': '1.06 MMac',
 'Mixer_params': '87.89 k'}

In [4]:
results

{'pamap2': {'dcl_macs': '401.66 MMac',
  'dcl_params': '745.43 k',
  'DeepSense_macs': '90.83 MMac',
  'DeepSense_params': '456.72 k',
  'GlobalFusion_macs': '69.18 MMac',
  'GlobalFusion_params': '355.41 k',
  'Attend_macs': '369.22 MMac',
  'Attend_params': '668.12 k',
  'ALAE_macs': '297.52 MMac',
  'ALAE_params': '413.52 k',
  'IfConv_macs': '140.26 MMac',
  'IfConv_params': '171.13 k',
  'CrossAttn_macs': '19.38 MMac',
  'CrossAttn_params': '124.88 k'},
 'dsads': {'dcl_macs': '576.15 MMac',
  'dcl_params': '1.63 M',
  'DeepSense_macs': '106.97 MMac',
  'DeepSense_params': '623.51 k',
  'GlobalFusion_macs': '108.61 MMac',
  'GlobalFusion_params': '431.89 k',
  'Attend_macs': '543.88 MMac',
  'Attend_params': '1.33 M',
  'ALAE_macs': '454.44 MMac',
  'ALAE_params': '855.96 k',
  'IfConv_macs': '205.82 MMac',
  'IfConv_params': '248.29 k',
  'CrossAttn_macs': '29.12 MMac',
  'CrossAttn_params': '187.8 k'},
 'dg': {'dcl_macs': '124.16 MMac',
  'dcl_params': '448.45 k',
  'DeepSense_ma

In [5]:
import pandas as pd
pd.DataFrame.from_dict(results).T.to_csv("flop2.csv")

In [12]:
    if sensor_channel//9==0:
        
        fusion = True
    else:
        fusion = False

True

In [11]:
model = If_ConvTransformer_W(1, sensor_channel, 32, 5, 3, 2, 64, 1, 0.2, dataset_name, window_length, nb_classes, fusion = fusion)

macs, params = get_model_complexity_info(model, (1,sensor_channel , window_length), as_strings=True,
                                       print_per_layer_stat=False, verbose=False)

Flops estimation was not finished successfully because ofthe following exception:
<class 'RuntimeError'> : Given groups=1, weight of size [32, 32, 1, 5], expected input[1, 1, 6, 200] to have 32 channels, but got 1 channels instead


Traceback (most recent call last):
  File "/home/kit/tm/px6680/miniconda3/envs/iswc23/lib/python3.9/site-packages/ptflops/pytorch_engine.py", line 62, in get_flops_pytorch
    _ = flops_model(batch)
  File "/home/kit/tm/px6680/.local/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1071, in _call_impl
    result = forward_call(*input, **kwargs)
  File "/pfs/data5/home/kit/tm/px6680/Conference/ISWC2023/I2S0W2C2_CFC/models/ifconv.py", line 411, in forward
    x = self.conv2(x)
  File "/home/kit/tm/px6680/.local/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1051, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/kit/tm/px6680/.local/lib/python3.9/site-packages/torch/nn/modules/container.py", line 139, in forward
    input = module(input)
  File "/home/kit/tm/px6680/.local/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1071, in _call_impl
    result = forward_call(*input, **kwargs)
  File "/home/kit/tm/px6680/.local/lib/python

TypeError: unsupported operand type(s) for //: 'NoneType' and 'int'

In [5]:
# {"dataset_name":"mhealth",  "sensor_channel":18, "window_length":128, #
#  "fft_window_length":8 , "nb_classes":12, "fft_feature_dim":32, #
#  "S_number_sensors_type":3,"L_sensor_locations":2}#

batch_size =1
feature_dim = 1
sensor_channel = 18
dataset_name = "Opportunity"
window_length = 128
nb_classes = 12

if sensor_channel%9==0:
    
    fusion = True
else:
    fusion = False
model = If_ConvTransformer_W(feature_dim, sensor_channel, 32, 5, 3, 2, 64, 1, 0.2, dataset_name, window_length, nb_classes, fusion = fusion)

input = torch.randn(1, 1,sensor_channel , window_length)
t = model(input)


macs, params = get_model_complexity_info(model, (1,sensor_channel , window_length), as_strings=True,
                                       print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       87.39 MMac
Number of parameters:           170.74 k


In [7]:
# {"dataset_name":"mhealth",  "sensor_channel":18, "window_length":128, #
#  "fft_window_length":8 , "nb_classes":12, "fft_feature_dim":32, #
#  "S_number_sensors_type":3,"L_sensor_locations":2}#

batch_size = 1
window_length = 128
sensor_channel = 18
ft_window_length = 8
ft_feature_dim = 32
hidden_dim = 24

model = CrossAttn(
    Ts_input_shape = (batch_size,1,window_length,sensor_channel),
    hidden_dim   = hidden_dim,
    FFT_input_shape = (batch_size,ft_feature_dim,ft_window_length,sensor_channel)
)

x_ts = torch.rand(batch_size, 1, window_length, sensor_channel)
x_ft = torch.rand(batch_size, ft_feature_dim, ft_window_length, sensor_channel)

y1,y2 = model([x_ts, x_ft])


def prepare_input(resolution):
    x1 = torch.FloatTensor(1, 1, window_length, sensor_channel)
    x2 = torch.FloatTensor(1, ft_feature_dim, ft_window_length, sensor_channel)
    return dict(x = [x1, x2])



macs, params = get_model_complexity_info(model, (1, window_length, sensor_channel), input_constructor=prepare_input, as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Warning! No positional inputs found for a module, assuming batch size is 1.
Computational complexity:       11.78 MMac
Number of parameters:           124.3 k 


In [3]:
batch_size =1
feature_dim = 1
window_length = 128
sensor_channel = 9

input_shape = (batch_size, feature_dim,window_length,sensor_channel)
num_class = 6
filter_scaling_factor=1

config = {  "nb_conv_blocks": 2,
  "nb_filters": 64,
  "dilation": 1,
  "batch_norm": 0,
  "filter_width": 7,
  "nb_layers_lstm": 1,
  "drop_prob": 0.2,
  "nb_units_lstm": 128}

model = DeepConvLSTM(input_shape, num_class, filter_scaling_factor,config=config)


x = torch.rand(batch_size, feature_dim, window_length, sensor_channel)

y =model(x)

macs, params = get_model_complexity_info(model, (feature_dim, window_length, sensor_channel), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       124.16 MMac
Number of parameters:           448.97 k


In [4]:
batch_size =1
feature_dim = 1
window_length = 128
sensor_channel = 9

input_shape = (batch_size, feature_dim,window_length,sensor_channel)
num_class = 6
filter_scaling_factor=1

config = {"hidden_dim": 128,
  "filter_num": 64,
  "filter_size": 5,
  "enc_num_layers": 2,
  "dropout": 0.5,
  "dropout_rnn": 0.25,
  "dropout_cls": 0.5,
  "activation": "ReLU",
  "sa_div": 1}

model = AttendDiscriminate(input_shape, num_class, config=config)


x = torch.rand(batch_size, feature_dim, window_length, sensor_channel)

y =model(x)

macs, params = get_model_complexity_info(model, (feature_dim, window_length, sensor_channel), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       120.55 MMac
Number of parameters:           445.38 k


In [5]:
batch_size =1
feature_dim = 1
window_length = 128
sensor_channel = 9
filter_nr = 64
nb_classes = 6

model = ALAE_TAE(input_shape=(batch_size, feature_dim, window_length, sensor_channel),      nb_classes=nb_classes)

x = torch.rand(batch_size, feature_dim, window_length, sensor_channel)

y =model(x)

macs, params = get_model_complexity_info(model, (feature_dim, window_length, sensor_channel), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
x torch.Size([1, 64, 128, 9])
torch.Size([1, 64, 128, 9])
Computational complexity:       98.77 MMac
Number of parameters:           265.29 k


In [4]:
batch_size =1
fft_feature_dim = 32
window_length = 128
sensor_channel = 18
S_number_sensors_type = 3
fft_segments_length = 16
nb_classes = 12
L_sensor_locations =2
model = GlobalFusion(
    input_shape=(batch_size, fft_feature_dim, window_length, sensor_channel),      
    S_number_sensors_type=S_number_sensors_type, 
    L_sensor_locations= L_sensor_locations, 
    fft_segments_length=fft_segments_length,
    nb_classes= nb_classes)

x = torch.rand(batch_size, 1, window_length, sensor_channel)

y =model(x)

macs, params = get_model_complexity_info(model, (1, window_length, sensor_channel), as_strings=True, print_per_layer_stat=False, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       38.76 MMac
Number of parameters:           333.64 k


In [ ]:
    {"dataset_name":"mhealth",  "sensor_channel":18, "window_length":128, #
     "fft_window_length":8 , "nb_classes":12, "fft_feature_dim":32, #
     "S_number_sensors_type":3,"L_sensor_locations":2}#
]
 'mhealth': {'dcl_macs': '241.26 MMac',
  'dcl_params': '744.65 k',
  'DeepSense_macs': '45.11 MMac',
  'DeepSense_params': '325.26 k',
  'GlobalFusion_macs': '38.76 MMac',
  'GlobalFusion_params': '333.64 k',
  'Attend_macs': '226.53 MMac',
  'Attend_params': '667.34 k',
  'ALAE_macs': '190.41 MMac',
  'ALAE_params': '413.13 k',
  'IfConv_macs': '87.39 MMac',
  'IfConv_params': '170.74 k',
  'CrossAttn_macs': '11.78 MMac',
  'CrossAttn_params': '124.3 k'}}

In [3]:
batch_size =1
fft_segments_length = 16
window_length = 128
sensor_channel = 18
k_number_sensors_group = 6
nb_classes =12
model = DeepSense(
    input_shape=(batch_size, 1, window_length, sensor_channel),  
    k_number_sensors_group=k_number_sensors_group, 
    fft_segments_length = fft_segments_length,
    nb_classes= nb_classes)

x = torch.rand(batch_size, 1, window_length, sensor_channel)

y =model(x)


macs, params = get_model_complexity_info(model, (1, window_length, sensor_channel), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       45.11 MMac
Number of parameters:           325.26 k


In [8]:
batch_size = 1
window_length = 128
sensor_channel = 9
ft_window_length = 10
ft_feature_dim = 32
hidden_dim = 32

model = CrossAttn(
    Ts_input_shape = (batch_size,1,window_length,sensor_channel),
    hidden_dim   = hidden_dim,
    FFT_input_shape = (batch_size,ft_feature_dim,ft_window_length,sensor_channel)
)

x_ts = torch.rand(batch_size, 1, window_length, sensor_channel)
x_ft = torch.rand(batch_size, ft_feature_dim, ft_window_length, sensor_channel)

y1,y2 = model([x_ts, x_ft])


def prepare_input(resolution):
    x1 = torch.FloatTensor(1, 1, window_length, sensor_channel)
    x2 = torch.FloatTensor(1, ft_feature_dim, ft_window_length, sensor_channel)
    return dict(x = [x1, x2])



macs, params = get_model_complexity_info(model, (1, window_length, sensor_channel), input_constructor=prepare_input, as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Warning! No positional inputs found for a module, assuming batch size is 1.
Computational complexity:       11.03 MMac
Number of parameters:           181.6 k 


In [3]:
input_length = 128
c_in = 9
num_classes = 12
filter_num = 24
model  = TinyHAR_Model((1,1, input_length, c_in ), 
                                         num_classes,
                                         filter_num = filter_num,#config["filter_num"],
                                         cross_channel_interaction_type = "attn",    # attn  transformer  identity
                                         cross_channel_aggregation_type = "FC",  # filter  naive  FC
                                         temporal_info_interaction_type = "lstm",     # gru  lstm  attn  transformer  identity
                                         temporal_info_aggregation_type = "tnaive")    # naive  filter  FC )

macs, params = get_model_complexity_info(model, (1, input_length, c_in), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       3.94 MMac
Number of parameters:           40.69 k 


In [5]:
input_length = 128
c_in = 9
num_classes = 12
filter_num = 5
fft_mixer_segments_length = 16


model = FFTMIXER_HAR_Model(
      input_shape=(1,1,input_length,c_in),
      number_class=num_classes,
      filter_num = filter_num,
      fft_mixer_segments_length = fft_mixer_segments_length,
      fft_mixer_share_flag=False,
      fft_mixer_temporal_flag=True,
      fft_mixer_FFT_flag=True,
      fft_mixer_layer_nr=2,
        expansion_factor = 0.2
)

macs, params = get_model_complexity_info(model, (1, input_length, c_in), as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

150 30
150 30
90 18
90 18
150 30
150 30
90 18
90 18
Computational complexity:       281.58 KMac
Number of parameters:           31.24 k 


In [8]:
8*9*8*9

5184

In [9]:
8*8*8*8

4096

In [ ]:
batch_size = 1
window_length = 128
sensor_channel = 9
ft_window_length = 10
ft_feature_dim = 32
hidden_dim = 32

model = CrossAttn(
    Ts_input_shape = (batch_size,1,window_length,sensor_channel),
    hidden_dim   = hidden_dim,
    FFT_input_shape = (batch_size,ft_feature_dim,ft_window_length,sensor_channel)
)

x_ts = torch.rand(batch_size, 1, window_length, sensor_channel)
x_ft = torch.rand(batch_size, ft_feature_dim, ft_window_length, sensor_channel)

y1,y2 = model([x_ts, x_ft])


def prepare_input(resolution):
    x1 = torch.FloatTensor(1, 1, window_length, sensor_channel)
    x2 = torch.FloatTensor(1, ft_feature_dim, ft_window_length, sensor_channel)
    return dict(x = [x1, x2])



macs, params = get_model_complexity_info(model, (1, window_length, sensor_channel), input_constructor=prepare_input, as_strings=True, print_per_layer_stat=False, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))